# Metisa - Data Science Challenge (Calvin)

Data: Anonymous Microsoft Web Data Data Set - http://archive.ics.uci.edu/ml/datasets/Anonymous+Microsoft+Web+Data

The data is in an ASCII-based sparse-data format called "DST". Each line of the data file starts with a letter which tells the line's type. The three line types of interest are Attribute, Case and Vote. Each Attribute is a website, each Case is a user and each Vote is an Attribute that the user visited. For more details, please read the data description file for the structure of the data set.

##Task:
1. Assuming we are at a time such that we only have the training data, we want to recommend websites that the users should visit based on their user ID (case ID number). Please construct a recommender system, train it with the training data set and then conduct recommendation for the users given the user ID.
2. Please also write the procedure to test your recommender with the test data set. Explain the metrics that you use for the testing.

##Please also answer the following questions:
1. What are the pros and cons of the recommendation algorithm you have used?
2. How did you evaluate your recommender's performance? Why?
3. Are you happy with you recommender's results? What could be a suitable baseline to compare your classifier's performance to?

##Remarks:
1. The challenge does not have 'the one' solution or answer. There are many ways to approach the task. Same holds true for the accompanying questions. Please motivate all the choices you have made.
2. We have stated the task with many implicit and explicit requirements. If you cannot comply with any of these requirements, please state this and work around.
3. We also value your input on how this challenge can be improved.
4. Very important: we want to see how you think. Please write down all your thoughts, however preliminary. We much prefer that you discuss an issue without offering a solution, rather than not mentioning it.


## Relevant Information:

    We created the data by sampling and processing the www.microsoft.com logs.
    The data records the use of www.microsoft.com by 38000 anonymous,
    randomly-selected users. For each user, the data lists all the areas of
    the web site (Vroots) that user visited in a one week timeframe.

    Users are identified only by a sequential number, for example, User #14988,
    User #14989, etc. The file contains no personally identifiable information.
    The 294 Vroots are identified by their title (e.g. "NetShow for PowerPoint")
    and URL (e.g. "/stream"). The data comes from one week in February, 1998.

    Dataset format:
	-- The data is in an ASCII-based sparse-data format called "DST".
           Each line of the data file starts with a letter which tells the line's type.
           The three line types of interest are:
               -- Attribute lines:
		     For example, 'A,1277,1,"NetShow for PowerPoint","/stream"'
                     Where:
                        'A' marks this as an attribute line,
                        '1277' is the attribute ID number for an area of the website
                                 (called a Vroot),
	                '1' may be ignored,
			'"NetShow for PowerPoint"' is the title of the Vroot,
                        '"/stream"' is the URL relative to "http://www.microsoft.com"
                -- Case and Vote Lines:
                    For each user, there is a case line followed by zero or more vote lines.
                     For example:
                           C,"10164",10164
                           V,1123,1
                           V,1009,1
                           V,1052,1
                      Where:
                         'C' marks this as a case line,
                          '10164' is the case ID number of a user,
                         'V' marks the vote lines for this case,
                         '1123', 1009', 1052' are the attributes ID's of Vroots that a
                                user visited.
                          '1' may be ignored.

Number of Instances:
      -- Training: 32711
      -- Testing:   5000
    Each instance represents an anonymous, randomly selected user of the web site.
Number of Attributes: 294
Attribute Information:
   Each attribute is an area ("vroot") of the www.microsoft.com web site.
   The datasets record which Vroots each user visited in a one-week timeframe
   in Feburary 1998.
Missing Attribute Values: The data is very sparse, so vroot visits are explicit,
    nonvisits are implicit (missing).
Class Distribution: 
    Mean number of vroot visits per case: 3.0

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_table('anonymous-msweb.data', skiprows = 7, sep = ',', header = None, names = ['Attribute', 'ID', 'Ig', 'Vroot', 'URL'])

/Users/calvincheung/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## First, divide the data into two tables: 
### 1) Attribute table - store all the attribute information
### 2) Cases and Votes tables - for extract information from our cases

In [3]:
# Attribute table
dfA = data.ix[0:293,:]
dfA.head()

,Attribute,ID,Ig,Vroot,URL
0,A,1287,1,International AutoRoute,/autoroute
1,A,1288,1,library,/library
2,A,1289,1,Master Chef Product Information,/masterchef
3,A,1297,1,Central America,/centroam
4,A,1215,1,For Developers Only Info,/developer


In [4]:
# The Case and Vote data is then store in dfC
dfC = data.ix[294:,:3]
dfC = dfC.reset_index()
dfC.ix[:10,:]

,index,Attribute,ID,Ig
0,294,C,10001,10001
1,295,V,1000,1
2,296,V,1001,1
3,297,V,1002,1
4,298,C,10002,10002
5,299,V,1001,1
6,300,V,1003,1
7,301,C,10003,10003
8,302,V,1001,1
9,303,V,1003,1


In [5]:
# The number of cases
dfC.loc[dfC.Attribute == 'C', 'index'].size

32711

In [6]:
# Show attribute and what they representing
dfA[dfA['ID'].isin([1000,1001,1002])]

,Attribute,ID,Ig,Vroot,URL
78,A,1001,1,Support Desktop,/support
217,A,1002,1,End User Produced View,/athome
268,A,1000,1,regwiz,/regwiz


# Recommender strategy:
1. Find relationships between Attribute, i.e. in the first case, ID:10001 visit site 1000, then visit 1001 and 1002, so it indicate that there is a relation between Support Desktop, End User Produced View and regwiz.
2. Formulate these relations from the case and vote data, then we can use these information to recommend website based on the history of website visited.
3. For Vroot that already visited, we remove it from the recommendation list.


In [7]:
# Extract the relationships from the data
relations = []
for i in range(0,dfC.ID.size-1):
    if dfC.Attribute[i] == 'V':
        for a in range(1,50):
            if dfC.Attribute[i+a] == 'V':
                relations.append([dfC.ID[i],dfC.ID[i+a]])
                relations.append([dfC.ID[i+a],dfC.ID[i]])
            if dfC.Attribute[i+a] == 'C':
                break
len(relations)

402738

In [8]:
# Put the relations into dataframe which is easier to work with
dfR = pd.DataFrame(relations, columns =['I','R'])
dfR.head()

,I,R
0,1000,1001
1,1001,1000
2,1000,1002
3,1002,1000
4,1001,1002


In [9]:
# Use value count function, we know which Vroot is most related to the initial Vroot.
Recommend = dfR.R[dfR.I == 1000].value_counts()
Recommend[:10]

# In this case, Vroot 1000 is most related to 1014, 1001 etc. 
# In other words, if the case visit Vroot 1000 (only), we should recommend 1014, 1001, 1018, 1008 & 1040

1014    373
1001    300
1018    254
1008    243
1040    236
1017    225
1034    216
1004    213
1003    156
1009    130
Name: R, dtype: int64

In [10]:
# If the case visit more than 1 Vroot, then the Recommend will add more conditions, i.e. 
Recommend = dfR.R[(dfR.I == 1000)|(dfR.I == 1014)|(dfR.I == 1036)].value_counts()
Recommend[:10]

# In this case, the visited Vroots are 1000, 1014 and 1036, 
# so the next recommendation will be Vroot 1040, 1008, 1017, 1018 & 1034

1040    979
1008    703
1017    649
1018    597
1034    566
1004    565
1001    554
1014    469
1000    459
1009    434
Name: R, dtype: int64

In [11]:
# Create a function that when input the case history (the list of Vroot that visited), based on the LAST 5 votes to make recommendation
def Rec_Vroot(vote):
    if len(vote) == 1:
        Recommend = dfR.R[(dfR.I == vote[-1])].value_counts()
    if len(vote) == 2:
        Recommend = dfR.R[(dfR.I == vote[-1])|(dfR.I == vote[-2])].value_counts()
    if len(vote) == 3:
        Recommend = dfR.R[(dfR.I == vote[-1])|(dfR.I == vote[-2])|(dfR.I == vote[-3])].value_counts()
    if len(vote) == 4:
        Recommend = dfR.R[(dfR.I == vote[-1])|(dfR.I == vote[-2])|(dfR.I == vote[-3])|(dfR.I == vote[-4])].value_counts()   
    if len(vote) == 5:
        Recommend = dfR.R[(dfR.I == vote[-1])|(dfR.I == vote[-2])|(dfR.I == vote[-3])|(dfR.I == vote[-4])|(dfR.I == vote[-5])].value_counts()
    if len(vote) > 5:
        Recommend = dfR.R[(dfR.I == vote[-1])|(dfR.I == vote[-2])|(dfR.I == vote[-3])|(dfR.I == vote[-4])|(dfR.I == vote[-5])].value_counts()
    X = list(Recommend.index[0:100])
    for x in range(0,5):
        for v in range(0,len(vote)):
            if X[x] == vote[v]:
                del X[x] # remove Vroot that already exist in history
    return X[:5]
    

In [12]:
# for example, if the case vote history is as follow: 1000,1001,1014,1025,1059,1043,1021,1032,1047
# Then the recommendation will be based on the history: 1059,1043,1021,1032,1047 instead of 
# the history: 1000,1001,1014,1025,1059'
clist = [1000,1001,1014,1025,1059,1043,1021,1032,1047]
Rec_Vroot(clist[:5])

[1018, 1004, 1003, 1008, 1034]

In [13]:
# E.g. for case #42650, the vote history is 1020,1017,1018,1004,1014,1035,1037,1009,1012,1003,1088,1034,1091,1167
clist = [1020,1017,1018,1004,1014,1035,1037,1009,1012,1003,1088,1034,1091,1167]
# Let's input the first 5 votes and let's see if the later come up in the result
# Test 1
print(Rec_Vroot(clist[:5]))
# In this case, the prediction 1009 is correct.

# Test 2
# Now input the first 8 votes (only the last 5 will use)
print(Rec_Vroot(clist[:8]))
# In this case, the prediction 1034 and 1003 are correct. 
# Note that 1009 is removed because it is in the prediction already.


[1008, 1034, 1001, 1009, 1003]
[1008, 1017, 1001, 1034, 1003]


# Procedure to test recommender strategy with the test data set

In this algorithm, we will recommend 5 Vroots and we want to know if they will match the case's 'future' record.

1) If our recommendation match the case's near future, that should score higher matching the far future.

2) If our higher priority recommendation match the case's future, that should score higher than our lower priority cases.

Based on the above criteria, the following function is created to measure the success rate of recommendation for a single prediction.

In [14]:
def R_score(prediction, target):
    success = []
    MPS_list = [1, 1.8, 2.4, 2.8, 3] # maximum score that based on target length
    MPS = MPS_list[len(target)-1]
    for p in range(0,len(prediction)):
        for t in range(0,len(target)):
            if prediction[p] == target[t]:
                rate = ((5-p) + (5-t))/10
                success.append(rate)
    score = sum(success)/MPS # the maximum score by this method is 3
    return score

In [15]:
A = [1,2,3,4,5]
B = [5]
R_score(A,B)

0.6

In [16]:
# Test 1
prediction = Rec_Vroot(clist[:5])
target = clist[6:11]
prediction, target
print(R_score(prediction, target))

# Test 2
prediction = Rec_Vroot(clist[:9])
target = clist[9:14]
prediction, target
print(R_score(prediction, target))

0.3
0.3666666666666667


In [17]:
# Make a sample to evaluation result
dfCase = dfC.loc[dfC.Attribute == 'C'][:4001]

Vroot_len = []
for i in range(0, dfCase.index.size-1):
    Vroot_len.append(dfCase.index[i+1] - dfCase.index[i] - 1)
    
dfCase['Vroot_len'] = float(0)
dfCase['Vroot_len'][:4000] = Vroot_len

dfCaseE = dfCase[dfCase.Vroot_len > 1]
dfC.index.size, dfCase.index.size, dfCaseE.index.size

/Users/calvincheung/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(131365, 4001, 2800)

In [18]:
# Generate a list of Vroot history for testing

dfCindex = dfC.loc[dfC.Attribute == 'C']
indexC = np.array(dfCindex.index)
indexC.size

VHist = []
Vlist = []
for i in range(0,4001):
    Cloc = indexC[i]
    Vlist.append(dfC.ID[Cloc+1])
    for a in range(2,50):
        if dfC.Attribute[Cloc+a] == 'V':
            Vlist.append(dfC.ID[Cloc+a])
        if dfC.Attribute[Cloc+a] == 'C':
            VHist.append(Vlist)
            Vlist = []
            break    
len(VHist)                

4001

In [19]:
# Only cases with at least 2 Vroot can perform any types of evaluation
LHist = []
for l in range(0,4001):
    if np.size(VHist[l]) > 1:
        LHist.append(VHist[l])
np.size(LHist)

2801

In [20]:
Lscore = []
Hscore = []
for i in range(0,2801):
    Clist = LHist[i]
    for l in range(1,50): #len(LHist[i])):
        target = Clist[l:l+5] #len(Clist)]
        if len(target) > 0:
            prediction = Rec_Vroot(Clist[:l])    
            Lscore.append(R_score(prediction,target))            
        if len(target) < 1:
            Hscore.append(Lscore)
            Lscore = []
            break
len(Hscore)

2801

In [21]:
# Vroot history for more than 2 entries
LHist[:10]

[[1000, 1001, 1002],
 [1001, 1003],
 [1001, 1003, 1004],
 [1003, 1004],
 [1008, 1009],
 [1010, 1000, 1011, 1012, 1013, 1014],
 [1015, 1016, 1017, 1018, 1019],
 [1020, 1021],
 [1025, 1026],
 [1027, 1017, 1026, 1028]]

In [22]:
# Prediction based on the first vote of each case
print(Rec_Vroot(LHist[0][0:1]),
      Rec_Vroot(LHist[1][0:1]),
      Rec_Vroot(LHist[2][0:1]),
      Rec_Vroot(LHist[3][0:1]),
      Rec_Vroot(LHist[4][0:1]))

[1014, 1001, 1018, 1008, 1040] [1018, 1003, 1004, 1008, 1017] [1018, 1003, 1004, 1008, 1017] [1001, 1018, 1004, 1008, 1035] [1034, 1009, 1018, 1017, 1004]


In [23]:
# Score
Hscore[:10]

[[0.5, 0.0],
 [0.9],
 [0.888888888888889, 0.9],
 [0.8],
 [0.9],
 [0.0, 0.17857142857142858, 0.25, 0.38888888888888884, 0.8],
 [0.6428571428571429, 0.6666666666666667, 0.3333333333333333, 0.0],
 [0.0],
 [1.0],
 [0.33333333333333337, 0.0, 0.0]]

In [24]:
# Overall mean score
# Convert list of lists into a single list
HscoreA = sum(Hscore, [])
np.mean(HscoreA)

0.50083157040407666

# Discussion:

## What are the pros and cons of the recommendation algorithm you have used?

In term of the database that form our recommendation statistics, it is based on the cases records, which consist of 32711 cases. The Pros would be that it is easy to update, e.g. when we have more data we can add them into the database so ideally we know which Vroot is most related to another Vroot. However, it is not tailor-made of each individual and the priority of our recommendation is only based on the public record. Furthermore, the relationship is based on one initial and one relation, but sometimes, a more powerful prediction would be two initials and one relation, this system has not been use in our database.

In term of formulating the recommendation, our strategy is based on the rating of the relationship. For example, if 1001 has a strong relationship with 1002, i.e. 1002 will be recommended when everytimes the case visit 1001 as long as it is in the last 5 entry in the history. Surely a weighting system on the history will be useful, i.e. Vroot at the last visit should weight higher than Vroot visit a while ago. 
    
## How did you evaluate your recommender's performance? Why?

In this algorithm, we will recommend 5 Vroots and we want to know if they will match the case's 'future' record. If our recommendation match the case's near future, that should score higher matching the far future. If our higher priority recommendation match the case's future, that should score higher than our lower priority cases. Based on this criteria, the score of each prediction will between 0 to 1, i.e. full point (1) will be given when the highest prority recommendation hit the next target. Or cases that we predict more than one future vote, normalise factors is applied.

Then, based on 4000 cases, there are 2800 cases with more than one entry, and we based on these cases to evaluate the performance, i.e. based on the past 5 votes (or available votes) to predict the next 5 votes (or available votes). The overall score is the average of every single predictions.
  
## Are you happy with you recommender's results? What could be a suitable baseline to compare your classifier's performance to?

The overall score is around 0.5, which is quite high considering other similar web prediction application such as Expedia (https://www.dataquest.io/blog/kaggle-tutorial/), however, the Expedia is a far more complicate exercise and the scoring system is much tougher, they need to predict the hotel booking but I only need to predict the next Vroot. Furthermore, the testing sample in this exercise is within the trainning set so it may overfit, too.


# Others
    
### The challenge does not have 'the one' solution or answer. There are many ways to approach the task. Same holds true for the accompanying questions. Please motivate all the choices you have made.

Another approach I am considering is related to the way how Vroot are weighted. When more than one vote in the history, the current method just use a multiple criteria to filter out the highest hits Vroot. But when one of the vote is highly popular (i.e. there are many high hits Vroots), then recommendation based on other vote may undermine. So this maybe the next exercise.

### We have stated the task with many implicit and explicit requirements. If you cannot comply with any of these requirements, please state this and work around.

Currently, this algorithm is not in the form that can use in application, it will still need a lot of work to make it usable, nevertheless, it has demonstrate the idea and evaluate the performance.

### We also value your input on how this challenge can be improved.

I think I did learn a lot by doing this exrcise, especially I never did any website recommendation data project works before. As a result, it is rather difficult to understand the problem at the start, such as what is Vroot, attribute etc. It could be better if there is a link to show how the website look like.
    
### Very important: we want to see how you think. Please write down all your thoughts, however preliminary. We much prefer that you discuss an issue without offering a solution, rather than not mentioning it.

I think this exercise is more related developing algorithm and using functions in pandas, but not much in machine learning. I have been thinking of how to apply machine learning to solve this problem, such as group the Vroot into clusters and then the recommendation would be based on which cluster that the past vote belong to. Such as the hits number to other Vroot can formulate a series and plot them in a graph. 
